# Lab 04- Radius of The Earth

### Lillian Hanson, 09/24/2025, GPGN318A

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#### Processing Calibrations
The relative gravimeters need to be calibrated to give us actual mGal values. The CG5 already accounts for this and the number it gives you is in units of mGals. For the LNR though, you must account for the calibration by hand, we have attached the (somewhat outdated) calibration sheet for our specific LNR meter. You can use this to calculate the relative gravity from the dial reading as follows:

Denote your dial reading as A.
In the supplied table (see attachment), find the nearest integer smaller than your reading from the column labeled “Counter Reading”, denote this as B.
Find the corresponding mGal value in the column labeled as “Value in mGal”, denote this as C.
Find the corresponding factor value from the column labeled as “Factor for Interval”, denote this as F.
Calculate the gravity in mGal using the formula: $g = C + F * (A - B)$.
Note: You might notice this equation as adjusting the gravity value using a linear interpolation between two known values.

In [5]:
df_lf = pd.read_csv('data/LNR_gravimeter(Sheet1).csv')
df_lf

,Location,Digital Reading,Times,Operator
0,1,3144.200,13:57,Anna Williams
1,1,3144.350,14:01,Ben Koller
2,1,3144.080,14:05,Zach Shallenberger
3,2,3142.640,14:30,Sam & Tessa & Tyna
4,3,3139.242,14:45,Krishna
5,3,3139.300,14:50,Joe
6,2,3142.830,14:59,Anna G
7,2,3142.705,15:04,Clara
8,1,3144.335,15:14,Alex
9,1,3144.245,15:19,Lillian


#### Measurement Averaging
Treat each instrument separately!

Average the gravity measurements taken at each visit to the stations. We visisted the stations in the following order:

- Station 1 : visit 1 
- Station 2 : visit 1
- Station 3 : visit 1
- Station 2 : visit 2
- Station 1 : visit 2
So you'll need to average all of the LNR readings at Station 1 : visit 1 together to obtain a single measurement for that location and time. Repeat with each of the visits to each station for each instrument (treat the two CG-5's separately).

At the end should have a single measurement for each of the three instruments at every visit: 5 for the LNR, 5 for CG-5 1 and 5 for CG-5 2.

In [9]:
column_names = [
    "LINE","STATION","ALT","GRAV","SD","TILTX","TILTY","TEMP","TIDE",
    "DUR","REJ","TIME","DEC_TIME_DATE","TERRAIN","DATE"
]

df_cg51 = pd.read_csv('data/CG5-1.TXT', sep=r'\s+', skiprows=34, header=None, names=column_names)
print(df_cg51.head())


   LINE  STATION      ALT      GRAV     SD  TILTX  TILTY  TEMP   TIDE  DUR  \
0   1.0      1.0  29.1014  3621.271  0.426    8.9   -4.5 -1.10  0.016   30   
1   1.0      1.0  30.0779  3621.424  0.459   -5.3   -5.9 -1.11  0.023   30   
2   1.0      1.0  30.0779  3621.361  0.135   -2.9   -7.6 -1.15  0.026   30   
3   1.0      1.0  29.8338  3621.371  0.791   -4.6   -1.6 -1.18  0.030   30   
4   1.0      2.0  28.8572  3619.895  0.217   -3.0   -4.0 -1.14  0.040   30   

   REJ      TIME  DEC_TIME_DATE  TERRAIN        DATE  
0    0  13:49:24    45886.57505      0.0  2025/09/18  
1    0  13:59:07    45886.58179      0.0  2025/09/18  
2    0  14:04:08    45886.58527      0.0  2025/09/18  
3    0  14:08:48    45886.58850      0.0  2025/09/18  
4    0  14:24:29    45886.59938      0.0  2025/09/18  


In [2]:
# Note these are currently just placeholders for whatever values you obtain
lnr_grav = np.array([1, 2, 3, 4, 5])
lnr_time = np.array([1, 2, 3, 4, 5])

cg5_1_grav = np.array([1, 2, 3, 4, 5])
cg5_1_time = np.array([1, 2, 3, 4, 5])

cg5_2_grav = np.array([1, 2, 3, 4, 5])
cg5_2_grav = np.array([1, 2, 3, 4, 5])

### Question 1
Comment on the variability of the measurements at each visit for each instrument.

#### Drift Correction
There are many things that could change the actual acceleration of gravity throughout the time of our survey. To correct for this, we assume that the change is linear in time and repeated the measurement at station 1 at the end of the survey (we made a loop).

Create a function that linearly interpolates the gravity values at Station 1 as a function of time:

In [3]:
def grav_interp(grav_1, grav_2, time_1, time_2, time):
    """Linearly interpolates as a function of time
    Parameters
    ----------
    grav_1, grav_2 : float
        The values of gravity at a single location at multiple times
    time_1, time_2 : float
        The times associated with grav_1 and grav_2, respectively
    time : float, numpy.ndarray
        The times you want to evaluate the interpolated gravity at a single location

    Returns
    -------
    float, numpy.ndarray
        The gravity at a single location evaluates at `time`.
    """
    return np.zeros_like(time) #TODO: some function of `time` linearly interpolating between station 1 visits 1 and 2.

You will need to calculate the value of gravity at station 1 at each time for each of the instruments. Then subtract that value from every measurement. If done correctly the repeated values of station 1 at the start and end should be the same and equal to 0.

In [4]:
lnr_drift_corr = lnr_grav - grav_interp(lnr_grav[0], lnr_grav[-1], lnr_time[0], lnr_time[1], lnr_time)
assert lnr_drift_corr[0] == lnr_drift_corr[-1] == 0

AssertionError: 

### Question 2
We repeated station 2 twice as well, how close are the values at this station after drift corrections?

#### Relative Gravity at each station
For each instrument you should now have their drift corrected relative gravity measurements. Since you have subtract the value of gravity of Station 1 from every Station, your measurements are relative to Station 1.

Next, for each instrument average any values at repeated stations to obtain a single measurement at each station (independant of time).

In [5]:
# These arrays should end up with your final relative
# values at each of the three stations for each instrument
lnr_rel = np.array([1, 2, 3])
cg5_1_rel = np.array([1, 2, 3])
cg5_2_rel = np.array([1, 2, 3])

#### Relative to Absolute
Both the CG5 and the LNR are relative gravimeters, meaning they do not measure the full gravity value, instead they measure differences in gravity between two stations. The units of their data are in mGal (after calibration), the difference between 1 mGal and another is accurate, but a single value itself is meaningless.

Ex. Imagine someone telling you it was 1.4233 degrees celsius warmer inside than it was outside. They obtained this number by using an instrument that read out 331$^\circ$C and 332.4233$^\circ$C. The value itself doesn’t mean anything, but the difference between them is accurate. If the true temperature inside was 24$^\circ$C, then you could use this hypothetical relative temperature sensor to determine that the temperature outside was actually 24.4233$^\circ$C.

Because we know for certain the gravity value at one of the stations, we can use its value, and the relative differences from the gravimeters, to determine the absolute gravity at each of the other stations.

After drift corrections your data are explicitly the differences between each visit and station 1 ($\Delta S_i$). Use these values to calculate the absolute gravity at each station.
$$\cdot g_{abs, S_2}= \Delta (S_2)+g_{abs, S_1}$$


In [7]:
lnr_absol = ...
cg5_1_absol = ...
cg5_2_absol = ...

### Question 3
For each instrument calculate an approximate gradient between stations 2 and 3.
$$\partial_zg \approx \frac{gs_3-gs_2}{\Delta H}$$ How close is this to the theoretical free air gradient of -0.3086 mGal/m? Comment on any sources of error you might have encountered.

#### Determining the radius of the Earth
With the absolute gravity values at each location we can now find the radius of the Earth, without needing to know the mass (or the gravitational constant)! We are going to use a first order Taylor series expansion of Newton’s Universal Law of Gravitation:

### Question 4
Find an expression for the radius of the Earth at Station 2.

First, find the derivative of Newton's law with respect to $R$
$$\frac{\partial g_{abs}}{\partial R} = ...$$
Then simplify this using $g_{abs}(R)$ itself. i.e.
$$\frac{\partial g_{abs}}{\partial R}=g_{abs}(R)...$$
Use Newton's law in the Taylor series with $f(a)=g_{abs}(S_2)$ and $f(x)=g_{abs}(S_3)$ including only the first order in $x$ to show that
$$g_{abs}(S_3) \approx g_{abs}(S_2)-2\frac{g(S_2)}{R_2}\Delta H$$
Finally solve for $R_2$ to obtain an expression for the radius of the Earth at Station 2. 

### Question 5
Use your equation to obtain estimates of the Earth for each instrument. Comment on how close they were to each other and how close they were to the distance to the center of the Earth in Golden. 

### Deliverables:
- Describe the steps necessary to take a reading with each gravimeter
- Answer the questions throughout the report (5 total)
- Present the results of your attempts to measure the radius of the Earth.
- Attatch your field notebook to your report.